In [3]:
from datasets import load_dataset, load_metric,Audio
import random
import pandas as pd
from IPython.display import display, HTML

import re

import json

from transformers import Wav2Vec2CTCTokenizer, Wav2Vec2FeatureExtractor, Wav2Vec2Processor, Wav2Vec2ForCTC, TrainingArguments, Trainer

import torchaudio

import numpy as np
import librosa

import IPython.display as ipd

import torch
from dataclasses import dataclass,field
from typing import Any,Dict,List,Optional,Union


c:\Users\Netra\Downloads\sdp project\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


FETCHING DATA FROM COMMON VOICE 

In [4]:
train_hindi = load_dataset('mozilla-foundation/common_voice_14_0', 'hi', split='train+validation', use_auth_token=True)
test_hindi = load_dataset('mozilla-foundation/common_voice_14_0', 'hi', split='test', use_auth_token=True)


train_hindi = train_hindi.cast_column('audio', Audio(sampling_rate=1600))
print(train_hindi[0]['audio'])

train_hindi = train_hindi.remove_columns(["accent","age","client_id","down_votes","gender","locale","segment","up_votes", 'variant'])
test_hindi = test_hindi.remove_columns(["accent","age","client_id","down_votes","gender","locale","segment","up_votes", 'audio', 'variant'])

c:\Users\Netra\Downloads\sdp project\.venv\Lib\site-packages\datasets\load.py:2508: FutureWarning: 'use_auth_token' was deprecated in favor of 'token' in version 2.14.0 and will be removed in 3.0.0.
You can remove this warning by passing 'token=<use_auth_token>' instead.
  warnings.warn(
c:\Users\Netra\Downloads\sdp project\.venv\Lib\site-packages\datasets\load.py:1454: FutureWarning: The repository for mozilla-foundation/common_voice_14_0 contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/mozilla-foundation/common_voice_14_0
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


{'path': 'C:\\Users\\Netra\\.cache\\huggingface\\datasets\\downloads\\extracted\\adfeeeb4da3177f3ea350cf284e8c0858c1d229d058a1ef876f57d885912d043\\hi_train_0/common_voice_hi_26008353.mp3', 'array': array([ 4.14504625e-10, -1.17346494e-10,  5.96340477e-10, ...,
       -2.40924619e-05,  4.24584359e-05, -5.74295773e-05]), 'sampling_rate': 1600}


FETCHING RANDOM SAMPLES

In [5]:
def show_random_elements(dataset, num_examples=10) :
    assert num_examples <= len(dataset), "Can't pick more elements than in dataset"
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks :
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    df = pd.DataFrame(dataset[picks])
    display(HTML(df.to_html()))
    
    
show_random_elements(train_hindi.remove_columns(["path"]), num_examples=20)

,audio,sentence
0,"{'path': 'C:\Users\Netra\.cache\huggingface\datasets\downloads\extracted\adfeeeb4da3177f3ea350cf284e8c0858c1d229d058a1ef876f57d885912d043\hi_train_0/common_voice_hi_27465863.mp3', 'array': [-7.216024187073344e-07, 6.549180397996679e-07, -5.308422146299563e-07, 3.984404202128644e-07, -1.7797283646814321e-07, -1.0402800398878753e-07, 4.6230337602537475e-07, -8.553284374102077e-07, 1.3411201962298946e-06, -1.8907622916231048e-06, 2.53041616815608e-06, -3.244143499614438e-06, 4.012767021777108e-06, -4.807334335055202e-06, 5.569193490373436e-06, -6.419022611225955e-06, 7.253443527588388e-06, -8.023455848160665e-06, 8.464459824608639e-06, -9.172854333883151e-06, 9.72005000221543e-06, -9.547501576889772e-06, 9.303800652560312e-06, -9.29221096157562e-06, 8.253326086560264e-06, -6.480279807874467e-06, 5.140213033882901e-06, -3.5632610888569616e-06, 5.991760190227069e-07, 3.018692950718105e-06, -5.984903054923052e-06, 9.680803486844525e-06, -1.4638315406045876e-05, 1.9267623429186642e-05, -2.3445492843165994e-05, 2.9114871722413227e-05, -3.463681787252426e-05, 3.955381180276163e-05, -4.4300861191004515e-05, 4.91751816298347e-05, -5.364422395359725e-05, 5.6791031965985894e-05, -5.8408710174262524e-05, 5.94042576267384e-05, -5.960101407254115e-05, 5.798606434836984e-05, -5.4655080020893365e-05, 4.977302887709811e-05, -4.14979258493986e-05, 3.119819666608237e-05, -1.8106728020939045e-05, 7.406495569739491e-06, 1.3840281098964624e-07, -9.585201041772962e-06, 8.024183625821024e-05, -0.00010503279918339103, 4.661834464059211e-05, -0.00012175305164419115, 0.0001772649266058579, -0.00017787606338970363, 0.00016105239046737552, -0.0002662332553882152, 0.00019730220083147287, -0.0004213219799567014, 0.00043676389032043517, -0.00022053623979445547, 0.00027230801060795784, -0.0015785067807883024, 0.0020593134686350822, 0.0033794566988945007, -0.0004479762283153832, -0.0002043575659627095, -0.0024315176997333765, -0.000419094692915678, 0.0011886853026226163, -0.00016387045616284013, -0.001077814493328333, -0.0006451619556173682, -0.0009343076962977648, 0.0015764081617817283, -0.0009780117543414235, 0.001337082707323134, 0.001383783295750618, 0.00010802829638123512, -0.0021018979605287313, 0.000330887152813375, 3.9030514017213136e-05, 1.064635580405593e-05, 0.002275645500048995, -0.0005213353433646262, 0.002234118524938822, -0.002452661283314228, -0.0011202216846868396, -0.002742139156907797, 0.001214926247484982, 0.001588794169947505, 4.93695551995188e-06, 0.002058630809187889, -0.000550096679944545, 0.0018589227693155408, ...], 'sampling_rate': 1600}",साबुन में मौजूद तत्वों से गर्भपात का खतरा
1,"{'path': 'C:\Users\Netra\.cache\huggingface\datasets\downloads\extracted\adfeeeb4da3177f3ea350cf284e8c0858c1d229d058a1ef876f57d885912d043\hi_train_0/common_voice_hi_27466228.mp3', 'array': [-2.063987913913934e-08, 4.4913253560707744e-08, -5.2149243146004665e-08, 2.5959241867212768e-08, -7.214698882762605e-08, 9.753110674637355e-08, -9.648390886241032e-08, 7.432829818299069e-08, -9.309587767347693e-08, 7.097747101170171e-08, -5.261862057182043e-08, -3.612169052757963e-08, 6.103569205606618e-08, -1.6900878563319566e-07, 5.778557010671648e-08, -6.782296964047418e-07, -5.026038820687972e-07, 1.3893435379941366e-06, -1.033171884046169e-06, 1.42412727655028e-06, 4.080744702150696e-07, -3.1177573873719666e-06, -7.164524049585452e-07, -2.287544702994637e-06, -2.041205249270206e-07, -1.208372395922197e-06, 2.434175712551223e-06, 4.946327294419461e-07, 4.4299881665210705e-06, 4.925978487335669e-07, 2.5688923415145837e-06, 3.5095149542030413e-07, -8.287412356366985e-08, -7.046040991554037e-06, 4.573585101752542e-06, -1.5799719221831765e-06, 8.397529427384143e-07, 5.779193998023402e-06, -4.901052761852043e-06, 4.23078336098115e-06, -7.833133167878259e-06, 4.369494490674697e-06, -6.476179351011524e-06, 2.2476147023553494e-06, -2.3192866137833335e-05, 8.08640197647037e-06, -8.895876817405224e-06, 8.64205230755033e-06, -1.875433213172073e-06, -1.4747191

SPECIAL SYMBOLS ELIMINATION

In [33]:
chars_to_ignore_regex = '[\,\?\.\!\-\;\:\"\%\'\*]'

def remove_special_char(batch) :
    batch['sentence'] = re.sub(chars_to_ignore_regex, '', batch['sentence']).lower() + " "
    return batch

train_hindi = train_hindi.map(remove_special_char)
test_hindi = test_hindi.map(remove_special_char)

print(train_hindi)
print(test_hindi)

Map: 100%|██████████| 2982/2982 [00:00<00:00, 17998.51 examples/s]

Dataset({
    features: ['path', 'audio', 'sentence'],
    num_rows: 6971
})
Dataset({
    features: ['path', 'sentence'],
    num_rows: 2982
})


In [34]:
def extract_all_chars(batch) :
    all_text = " ".join(batch['sentence'])
    vocab = list(set(all_text))
    return {'vocab': [vocab], "all_text": [all_text]}

vocab_train = train_hindi.map(extract_all_chars, batched=True, batch_size=-1, keep_in_memory=True, remove_columns=train_hindi.column_names)
vocab_test = test_hindi.map(extract_all_chars, batched=True, batch_size=-1, keep_in_memory=True, remove_columns=test_hindi.column_names)
print(vocab_train)

Map: 100%|██████████| 2982/2982 [00:00<00:00, 85551.20 examples/s]

Dataset({
    features: ['vocab', 'all_text'],
    num_rows: 1
})


PADDING UNKNOWN VALUES 

In [35]:
vocab_list = list(set(vocab_train['vocab'][0]) | set(vocab_test['vocab'][0]))
vocab_dict = {v: k for k, v in enumerate(vocab_list)}
vocab_dict

vocab_dict['|'] = vocab_dict[" "]
del vocab_dict[" "]

vocab_dict["[UNK]"] = len(vocab_dict)
vocab_dict["[PAD]"] =len(vocab_dict)
len(vocab_dict)


104

STORING VOCAB LIST IN JSON FILE

In [9]:
with open('vocab.json', 'w') as vocab_file : 
    json.dump(vocab_dict, vocab_file)
    

CREATING FEATURE EXTRACTOR

In [10]:
tokenizer = Wav2Vec2CTCTokenizer('./vocab.json', unk_token="[UNK]", pad_token="[PAD]", word_delimiter_token="|")

feature_extractor = Wav2Vec2FeatureExtractor(feature_size=1, sampling_rate=1600, padding_value=0.0, do_normalize=True, return_attention_mask=True)

processor = Wav2Vec2Processor(feature_extractor=feature_extractor, tokenizer=tokenizer)


processor.save_pretrained('E:\\SDP\\pretrained\\hindi')

[]

PREPROCESSING

In [11]:
# mapped all the audio files successfully 
def speech_file_to_array_fn(batch):
    speech_array,sampling_rate = torchaudio.load(batch["path"])
    batch["speech"]=speech_array[0].numpy()
    batch['sampling_rate'] = sampling_rate
    batch["target_text"]=batch["sentence"]
    return batch

vocab_train = train_hindi.map(speech_file_to_array_fn, remove_columns=train_hindi.column_names)
vocab_test = test_hindi.map(speech_file_to_array_fn, remove_columns=test_hindi.column_names)

RESAMPLING AUDIO FROM 48kHZ TO 16kHZ 

In [12]:
def resampling(batch):
    batch["speech"] = librosa.resample(y=np.asarray(batch["speech"]), orig_sr=48_00, target_sr=16_00)
    batch["sampling_rate"] = 16_00
    return batch

common_voice_train = vocab_train.map(resampling)
common_voice_test = vocab_test.map(resampling)
print(common_voice_train)

Dataset({
    features: ['speech', 'sampling_rate', 'target_text'],
    num_rows: 6971
})


In [13]:
rand_int = random.randint(0, len(common_voice_train))
ipd.Audio(data=np.asarray(common_voice_train[rand_int]['speech']), autoplay=True, rate=16000)

PRINTING TEXT

In [14]:
rand_int = random.randint(0, len(common_voice_train))
print('target text : ', common_voice_train[rand_int]['target_text'])
print('input array shape : ', np.asarray(common_voice_train[rand_int]['speech']).shape)
print('sampling rate : ', common_voice_train[rand_int]['sampling_rate'])

print(common_voice_train)

target text :  सलमान खान के कहने पर सिद्धू ने कपिल के शो से किया किनारा 
input array shape :  (53376,)
sampling rate :  1600
Dataset({
    features: ['speech', 'sampling_rate', 'target_text'],
    num_rows: 6971
})


PREPARING DATASET FOR TRAINING

In [15]:
def prepare_dataset(batch) :
    assert(
        len(set(batch['sampling_rate'])) == 1
    ), f"Make sure all input have same sampling rate of {processor.feature_extractor.sampling_rate}"
    batch['input_values'] = processor(batch['speech'], sampling_rate = batch['sampling_rate'][0]).input_values
    with processor.as_target_processor():
        batch['labels'] = processor(batch['target_text']).input_ids
    return batch

common_voice_train = common_voice_train.map(prepare_dataset, remove_columns=common_voice_train.column_names, batch_size=8, batched=True)
common_voice_test = common_voice_test.map(prepare_dataset, remove_columns= common_voice_test.column_names, batch_size=8, batched=True)

Map:   0%|          | 0/6971 [00:00<?, ? examples/s]c:\Users\Netra\Downloads\sdp project\.venv\Lib\site-packages\transformers\models\wav2vec2\processing_wav2vec2.py:156: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warnings.warn(
Map: 100%|██████████| 2982/2982 [01:43<00:00, 28.74 examples/s]


SETUP TRAINER

In [28]:
@dataclass

class DataCollatorCTCWithPadding :
    processor : Wav2Vec2Processor
    padding : Union[bool, str] = True
    max_length : Optional[int] = None
    max_length_labels : Optional[int] = None
    pad_to_multiple_of : Optional[int] = None
    pad_to_multiple_of_labels : Optional[int] = None

    def __call__(self, features : List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor] :
        input_features = [{'input_values' : feature['input_values']} for feature in features]
        label_features = [{'input_ids' : feature['labels']} for feature in features]
        batch =self.processor.pad(
            input_features,
            padding= self.padding,
            max_length= self.max_length_labels,
            pad_to_multiple_of= self.pad_to_multiple_of_labels,
            return_tensors= "pt",
            
            
        )               
        with self.processor.as_target_processor():
            labels_batch= self.processor.pad(
                label_features,
                padding= self.padding,
                max_length= self.max_length_labels,
                pad_to_multiple_of= self.pad_to_multiple_of_labels,
                return_tensors= "pt",
                
            )
        labels= labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1),-100)
        batch["labels"]=labels
        
        return batch
    
data_collator= DataCollatorCTCWithPadding(processor=processor,padding=True)
print(data_collator)

DataCollatorCTCWithPadding(processor=Wav2Vec2Processor:
- feature_extractor: Wav2Vec2FeatureExtractor {
  "do_normalize": true,
  "feature_extractor_type": "Wav2Vec2FeatureExtractor",
  "feature_size": 1,
  "padding_side": "right",
  "padding_value": 0.0,
  "processor_class": "Wav2Vec2Processor",
  "return_attention_mask": true,
  "sampling_rate": 1600
}

- tokenizer: Wav2Vec2CTCTokenizer(name_or_path='', vocab_size=103, model_max_length=1000000000000000019884624838656, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '[UNK]', 'pad_token': '[PAD]'}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
	102: AddedToken("[UNK]", rstrip=True, lstrip=True, single_word=False, normalized=False, special=False),
	103: AddedToken("[PAD]", rstrip=True, lstrip=True, single_word=False, normalized=False, special=False),
	104: AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=False, sp

WORD ERROR RATE IN ASR MODEL

In [17]:
wer_metric = load_metric('wer')
print(wer_metric)

C:\Users\Netra\AppData\Local\Temp\ipykernel_20716\3366784470.py:1: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  wer_metric = load_metric('wer')


c:\Users\Netra\Downloads\sdp project\.venv\Lib\site-packages\datasets\load.py:753: FutureWarning: The repository for wer contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.17.0/metrics/wer/wer.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


Metric(name: "wer", features: {'predictions': Value(dtype='string', id='sequence'), 'references': Value(dtype='string', id='sequence')}, usage: """
Compute WER score of transcribed segments against references.

Args:
    references: List of references for each speech input.
    predictions: List of transcriptions to score.
    concatenate_texts (bool, default=False): Whether to concatenate all input texts or compute WER iteratively.

Returns:
    (float): the word error rate

Examples:

    >>> predictions = ["this is the prediction", "there is an other sample"]
    >>> references = ["this is the reference", "there is another one"]
    >>> wer = datasets.load_metric("wer")
    >>> wer_score = wer.compute(predictions=predictions, references=references)
    >>> print(wer_score)
    0.5
""", stored examples: 0)


COMPUTING METRIC USING LOGIT VECTOR

In [27]:
def compute_metrics(pred) :
    pred_logits = pred.predictions
    pred_ids = np.argmax(pred_logits, axis=-1)
    pred.label_ids[pred.label_ids == -100] = processor.tokenizer.pad_token_id
    pred_str = processor.batch_decode(pred_ids)
    label_str = processor.batch_decode(pred.label_ids, group_tokens=False)
    wer = wer_metric.compute(predictions= pred_str, references=label_str)
    
    correct_predictions = sum(1 for pred, label in zip(pred_str, label_str) if pred == label)
    total_samples = len(label_str)
    accuracy = correct_predictions / total_samples
    return {'wer' : wer}


HYPER PARAMETER TUNNING

In [19]:
import tensorflow as tf
model = Wav2Vec2ForCTC.from_pretrained(
    'facebook/wav2vec2-large-xlsr-53',
    attention_dropout = 0.1,
    hidden_dropout = 0.1, 
    feat_proj_dropout = 0.0, 
    mask_time_prob = 0.05,
    layerdrop = 0.1,
    gradient_checkpointing = True,
    ctc_loss_reduction = "mean",
    pad_token_id = processor.tokenizer.pad_token_id,
    vocab_size = len(processor.tokenizer)
)
model.freeze_feature_encoder()
print(model)


Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-large-xlsr-53 and are newly initialized: ['lm_head.bias', 'lm_head.weight', 'wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original0', 'wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original1']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Wav2Vec2ForCTC(
  (wav2vec2): Wav2Vec2Model(
    (feature_extractor): Wav2Vec2FeatureEncoder(
      (conv_layers): ModuleList(
        (0): Wav2Vec2LayerNormConvLayer(
          (conv): Conv1d(1, 512, kernel_size=(10,), stride=(5,))
          (layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
          (activation): GELUActivation()
        )
        (1-4): 4 x Wav2Vec2LayerNormConvLayer(
          (conv): Conv1d(512, 512, kernel_size=(3,), stride=(2,))
          (layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
          (activation): GELUActivation()
        )
        (5-6): 2 x Wav2Vec2LayerNormConvLayer(
          (conv): Conv1d(512, 512, kernel_size=(2,), stride=(2,))
          (layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
          (activation): GELUActivation()
        )
      )
    )
    (feature_projection): Wav2Vec2FeatureProjection(
      (layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
      (projec

In [24]:
training_args = TrainingArguments(
    output_dir= r'E:\\SDP\\pretrained\\hindi',
    group_by_length= True,
    per_device_train_batch_size= 4,
    per_device_eval_batch_size=4,
    weight_decay=0.01,
    metric_for_best_model='accuracy',
    greater_is_better=True,
    dataloader_num_workers=0,
    gradient_accumulation_steps= 8,
    evaluation_strategy= 'epoch',
    num_train_epochs= 10,
    save_steps= 5000, 
    eval_steps= 100,
    logging_steps= 100,
    learning_rate= 1e-5,
    #warmup_steps= 500,
    save_total_limit= 2,
)
print(training_args)
trainer = Trainer(
    model= model,
    data_collator= data_collator,
    args= training_args,
    compute_metrics= compute_metrics,
    train_dataset = common_voice_train,
    eval_dataset = common_voice_test,
    tokenizer = processor.feature_extractor,
)
print(trainer)



TrainingArguments(
_n_gpu=0,
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
auto_find_batch_size=False,
bf16=False,
bf16_full_eval=False,
data_seed=None,
dataloader_drop_last=False,
dataloader_num_workers=0,
dataloader_persistent_workers=False,
dataloader_pin_memory=True,
ddp_backend=None,
ddp_broadcast_buffers=None,
ddp_bucket_cap_mb=None,
ddp_find_unused_parameters=None,
ddp_timeout=1800,
debug=[],
deepspeed=None,
disable_tqdm=False,
dispatch_batches=None,
do_eval=True,
do_predict=False,
do_train=False,
eval_accumulation_steps=None,
eval_delay=0,
eval_steps=100,
evaluation_strategy=IntervalStrategy.EPOCH,
fp16=False,
fp16_backend=auto,
fp16_full_eval=False,
fp16_opt_level=O1,
fsdp=[],
fsdp_config={'min_num_params': 0, 'xla': False, 'xla_fsdp_grad_ckpt': False},
fsdp_min_num_params=0,
fsdp_transformer_layer_cls_to_wrap=None,
full_determinism=False,
gradient_accumulation_steps=8,
gradient_checkpointing=False,
gradient_checkpointing_kwargs=None,
greater_is_better

TRAINING

In [ ]:
trainer.train()

#after trainining we will evaluate 
# results = trainer.evaluate()
# print(results)

# Access the CTC loss from training logs
# train_logs = trainer.state.log_history
# train_ctc_loss = [log['loss'] for log in train_logs if 'loss' in log.keys()]
# print("Training CTC Loss:", train_ctc_loss)

# Access the CTC loss from evaluation results
# eval_results = trainer.evaluate()
# eval_ctc_loss = eval_results['eval_loss']
# print("Evaluation CTC Loss:", eval_ctc_loss)